Installation of necessary packages (and creation of a conda env):
```
./install_main.sh 
```

Process results obtained via running command:
```
bash -i run_qa_main.sh
```

In [1]:
SEED_QTY=3
OUTPUT_PREF='results_qa/output_res_'
#OUTPUT_PREF='../test_accelerate/results/output_res_'

In [2]:
#sync_step_arr = [1, 2, 4, 8, 16]
sync_step_arr = [1, 2, 4, 8, 16, 32, 64, 128, 256]

res_keys = ['1gpu'] + \
        [f'accum_steps_{k}' for k in sync_step_arr] + \
        [f'nosync_steps_{k}' for k in sync_step_arr]

In [3]:
import json

res_dict = {k : [] for k in res_keys}

for train_qty in [4000, 40000]:
    for rk in res_keys:
        one_res = {}
        
        for seed in range(SEED_QTY):
            tmp = json.load(open(f'{OUTPUT_PREF}{train_qty}_{rk}/{seed}/all_results.json'))
            if seed == 0:
                for mk in tmp.keys():
                    one_res[mk] = []

            for mk in tmp.keys():
                one_res[mk].append(tmp[mk])
                 
        res_dict[rk].append(one_res)
    

In [4]:
res_dict['1gpu']

[{'eval_exact_match': [69.65941343424787,
   69.78240302743615,
   69.57426679280984],
  'eval_f1': [80.26637915296253, 80.08236129993176, 79.77908879505684],
  'eval_train_time': [292.7783579826355, 293.6336350440979, 292.957053899765]},
 {'eval_exact_match': [80.88930936613056, 81.2961210974456, 81.20151371807],
  'eval_f1': [88.4562154274226, 89.04475780185317, 89.05548149018428],
  'eval_train_time': [2731.3763070106506,
   2731.3531460762024,
   2732.5005860328674]}]

In [5]:
tmp

{'eval_exact_match': 0.8325449385052034,
 'eval_f1': 8.960882770182382,
 'eval_train_time': 701.0928683280945}

In [6]:
import numpy as np

res_dict_str = {k : [] for k in res_keys}

for cidx, train_qty in enumerate([4000, 40000]):
    for rk in res_keys:
        speed_up = round(np.mean(res_dict['1gpu'][cidx]['eval_train_time']) / np.mean(res_dict[rk][cidx]['eval_train_time']), 1)
        f1 = str(round(np.mean(res_dict[rk][cidx]['eval_f1']), 2)) +'/' +str(round(np.max(res_dict[rk][cidx]['eval_f1']), 2))
        #print(rk, speed_up, f1)
        if rk != '1gpu':
            res_dict_str[rk].append(f'f1={f1} {speed_up}x')
        else:
            res_dict_str[rk].append(f'f1={f1}')  
            
            
res_dict_str

{'1gpu': ['f1=80.04/80.27', 'f1=88.85/89.06'],
 'accum_steps_1': ['f1=79.3/79.81 1.5x', 'f1=32.83/87.76 1.4x'],
 'accum_steps_2': ['f1=79.96/80.13 2.2x', 'f1=88.34/88.49 2.2x'],
 'accum_steps_4': ['f1=76.85/77.8 3.0x', 'f1=87.93/88.68 3.0x'],
 'accum_steps_8': ['f1=75.48/75.85 3.8x', 'f1=88.05/88.48 3.8x'],
 'accum_steps_16': ['f1=64.94/67.89 4.3x', 'f1=87.53/87.99 4.4x'],
 'accum_steps_32': ['f1=19.43/30.17 4.6x', 'f1=86.61/87.06 4.8x'],
 'accum_steps_64': ['f1=10.74/12.96 4.9x', 'f1=84.1/84.54 5.0x'],
 'accum_steps_128': ['f1=8.31/9.3 5.0x', 'f1=58.5/76.43 5.1x'],
 'accum_steps_256': ['f1=8.24/8.79 5.0x', 'f1=45.13/58.2 5.2x'],
 'nosync_steps_1': ['f1=79.99/80.34 1.2x', 'f1=88.65/88.79 1.1x'],
 'nosync_steps_2': ['f1=80.28/80.49 1.8x', 'f1=88.52/88.57 1.8x'],
 'nosync_steps_4': ['f1=80.16/80.5 2.4x', 'f1=88.75/88.99 2.4x'],
 'nosync_steps_8': ['f1=80.34/80.65 3.0x', 'f1=88.87/89.17 3.0x'],
 'nosync_steps_16': ['f1=80.15/80.34 3.4x', 'f1=88.58/88.83 3.3x'],
 'nosync_steps_32': ['f1=79

In [7]:
import pandas as pd
df=pd.DataFrame.from_dict(res_dict_str)

In [8]:
df

,1gpu,accum_steps_1,accum_steps_2,accum_steps_4,accum_steps_8,accum_steps_16,accum_steps_32,accum_steps_64,accum_steps_128,accum_steps_256,nosync_steps_1,nosync_steps_2,nosync_steps_4,nosync_steps_8,nosync_steps_16,nosync_steps_32,nosync_steps_64,nosync_steps_128,nosync_steps_256
0,f1=80.04/80.27,f1=79.3/79.81 1.5x,f1=79.96/80.13 2.2x,f1=76.85/77.8 3.0x,f1=75.48/75.85 3.8x,f1=64.94/67.89 4.3x,f1=19.43/30.17 4.6x,f1=10.74/12.96 4.9x,f1=8.31/9.3 5.0x,f1=8.24/8.79 5.0x,f1=79.99/80.34 1.2x,f1=80.28/80.49 1.8x,f1=80.16/80.5 2.4x,f1=80.34/80.65 3.0x,f1=80.15/80.34 3.4x,f1=79.59/79.96 3.6x,f1=79.68/79.94 3.8x,f1=79.21/79.85 3.9x,f1=79.21/79.85 3.9x
1,f1=88.85/89.06,f1=32.83/87.76 1.4x,f1=88.34/88.49 2.2x,f1=87.93/88.68 3.0x,f1=88.05/88.48 3.8x,f1=87.53/87.99 4.4x,f1=86.61/87.06 4.8x,f1=84.1/84.54 5.0x,f1=58.5/76.43 5.1x,f1=45.13/58.2 5.2x,f1=88.65/88.79 1.1x,f1=88.52/88.57 1.8x,f1=88.75/88.99 2.4x,f1=88.87/89.17 3.0x,f1=88.58/88.83 3.3x,f1=88.5/88.6 3.6x,f1=88.51/88.63 3.8x,f1=88.36/88.39 3.8x,f1=33.66/88.05 3.9x


In [9]:
#df.to_csv('results.tsv')

In [10]:
#!cat results.tsv